# CKP8087 - Estrutura de Dados
<img  src="https://img.shields.io/badge/UFC-CKP9011-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" />

*Vaux Gomes*

## Parte I - Preparação dos Dockers

#### Pre-requisitos
 - Docker
 - Docker Compose ou Plugin

#### Comandos

```sh
$ cd /path/to/work
$ cat > docker-compose.yml << 'EOF'
service:
  postgres:
    image: postgres
    container_name: postgres
    ports:
      - 5432:5432
    expose:
      - 5432
    networks:
      - local-network
    environment:
      - POSTGRES_USER=root
      - POSTGRES_PASSWORD=root
      - POSTGRES_DB=postgres

  neo4j:
    image: neo4j:latest
    container_name: neo4j
    ports:
      - "7474:7474"   # Neo4j Browser
      - "7687:7687"   # Bolt protocol
    networks:
      - local-network
    environment:
      - NEO4J_AUTH=neo4j/12345678             # Set do username/password
    volumes:
      - ./Neo4j/data:/data                    # Pasta para armazenar os dados
      - ./Neo4j/logs:/logs                    # Pasta para armazenar os logs
      - ./Neo4j/import:/var/lib/neo4j/import  # Pasta para importe de CSV
      - ./Neo4j/plugins:/plugins              # Pasta dos plugins
      - ./Neo4j/dumps:/dumps                  # Pasta para os dumps

networks:
  local-network:
    driver: bridge
EOF
```

<div class="alert alert-block alert-info">
    [OPCIONAL]: Escolha um nome melhor para o banco
</div>

## Parte II - Restauração do Dump

```sh
# Start do container
$ cd /path/to/work
$ docker-compose -up -d postgres

# Restore do dump
$ docker exec -i postgres psql -U root -d postgres < /path/to/backup.dump
```

![](./files/images/dbeaver.png)

## Parte III - Importando e tradandos os dados

In [156]:
# Installs - Necessário porque meu Jupyter está em um Docker
!pip install pandas  
!pip install sqlalchemy
!pip install psycopg2-binary       # Drive do postgress 

# Imports
import numpy as np
import pandas as pd
import psycopg2 as pg

In [157]:
# DB Config
username  = 'root'
password  = 'root'
host      = 'postgres' # NOME DO CONTAINER
port      = '5432'
database  = 'postgres' # NOME DO DATABASE

# Connection
conn = pg.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    dbname=database
)

# Dataframe
query = '''
SELECT 
    id_member_anonymous as member,  
    id_group_anonymous as group,
    text_content_anonymous as text,
    score_sentiment, score_misinformation
FROM tb_whatsapp_messages WHERE trava_zap = false'''
df = pd.read_sql(query, conn)

conn.close()

# Info
rows = df.shape[0]

# Data
print(df.shape)
df.head()

/tmp/ipykernel_86/364225286.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(407911, 5)


,member,group,text,score_sentiment,score_misinformation
0,eacc81d81047368e08bdcee59a0e69e2,970fc18f0d5608107b7822a2adbac3f8,None,NaN,NaN
1,542d038bf37b9f9871d6e8dac6fd4230,589e16e85b442fa82e8e0061fa2731e6,Vou ali,0.0000,NaN
2,3a8e41b9e1da548ef0acd0a57b398da4,e110071613239754d38878f7e046e95b,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867
3,3a8e41b9e1da548ef0acd0a57b398da4,7ee4235534ec624ebd61373b87ad8c20,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867
4,3a8e41b9e1da548ef0acd0a57b398da4,ee85f63c945ffa50ba8bb57acf2c1bf9,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867


### Filtros & Flags

In [158]:
# Filtro do número de palavras (5+)
df = df[df.text.str.count('\s').gt(3)] # 4 espaços ou mais
print(f'Reduzido para: {df.shape} ({(df.shape[0]/rows)*100:.2f}% do tamanho original)')

Reduzido para: (186316, 5) (45.68% do tamanho original)


In [159]:
# Flag de viral
counts = df['text'].value_counts()
df['viral'] = df['text'].isin(counts[counts > 1].index)
print(f'Reduzido para: {df[df.viral == True].shape} ({(df[df.viral == True].shape[0]/rows)*100:.2f}% do tamanho)')

Reduzido para: (110091, 6) (26.99% do tamanho)


In [160]:
# Flaf de misinformation
df['misinformation'] = df.score_misinformation >= 0.7
print(f'Reduzido para: {df[df.misinformation == True].shape} ({(df[df.misinformation == True].shape[0]/rows)*100:.2f}% do tamanho)')

Reduzido para: (8749, 7) (2.14% do tamanho)


### Grafos

In [161]:
def build(df, v=False):
    '''
    Função para mapeamento do dataframe em uma estrutura de contagem de mensagens
    
    graph = {
        'senders': {
            'sender_1': {
                'msgs': {
                    'receiver_1': Integer,
                    'receiver_2': Integer
                },
                stats: {
                    'geral': {
                        'grau_centralidade': Integer,     # Quantidade de ARESTAS (muda de acordo com o conjunto de dados apenas)
                        'forca': Integer                  # Quantidade de MENSAGENS
                    },
                    'viral': {
                        'grau_centralidade': Integer,
                        'forca': Integer
                    },
                    'misinformation': {
                        'grau_centralidade': Integer,
                        'forca': Integer
                    },
                }
            }...
        },
        arestas: Integer
    }
    '''
    
    graph = {
        'senders': {},
        'stats': {
            'arestas': 0,
            'nos': df.member.unique().size,
            'geral': df.shape[0],
            'viral': df[df['viral']].shape[0],
            'misinformation': df[df['misinformation']].shape[0]   
        }
    }

     #Monte uma tabela contendo a quantidade de nós e a quantidade de arestas para cada grafo
    #(mensagens gerais, mensagens virais e mensagens com desinformação).

    
    for g in df.group.unique()[:]:
        dfg = df[df.group == g]                       # Group filter
        members_ = dfg.member.value_counts()          # Group Member Counts
        
        for s, msgs in members_.items():              # s: Sender
            dfgs = dfg[dfg.member == s]               # (GROUP, SENDER) Messages
            
            # Node
            graph['senders'][s] = graph['senders'].get(s, {
                'msgs': {}, 
                'stats': {
                    'geral': {
                        'grau_centralidade': 0,
                        'forca': 0
                    },
                    'viral': {
                        'grau_centralidade': 0,
                        'forca': 0
                    },
                    'misinformation': {
                        'grau_centralidade': 0,
                        'forca': 0
                    }, 
                }
            })

            # Node statuses: Geral
            graph['senders'][s]['stats']['geral']['grau_centralidade'] += members_.shape[0] - 1
            graph['senders'][s]['stats']['geral']['forca'] += msgs

            # Node statuses: Viral
            graph['senders'][s]['stats']['viral']['grau_centralidade'] += members_.shape[0] - 1
            graph['senders'][s]['stats']['viral']['forca'] += dfgs[dfgs['viral']].shape[0]

            # Node statuses: Misinformation
            graph['senders'][s]['stats']['misinformation']['grau_centralidade'] += members_.shape[0] - 1
            graph['senders'][s]['stats']['misinformation']['forca'] += dfgs[dfgs['misinformation']].shape[0]

            # Listing receivers
            for r in members_.keys():                 # r: Receiver
                if s == r:
                    continue
                    
                graph['senders'][s]['msgs'][r] = graph['senders'][s]['msgs'].get(r, 0) + msgs
                graph['stats']['arestas'] += msgs

    #
    members = df.member.unique()
    groups = df.group.unique()
    
    # Verbose
    if v:
        print('- Total de mensagens:', df.shape[0])
        print('- Número de membros:\n  - Ativos:', len(graph['senders']))
        print('  - Total:', len(members))
        print('- Número de grupos:', len(groups))
        print()
        
        
    return graph, members, groups

#### Grafo Geral

In [162]:
%time geral, g_members, _ = build(df, v=True)

- Total de mensagens: 186316
- Número de membros:
  - Ativos: 6165
  - Total: 6165
- Número de grupos: 237

CPU times: user 7.86 s, sys: 6.49 ms, total: 7.87 s
Wall time: 7.87 s


#### Grafo Viral

> Considerando viral se houver uma mensagem que foi enviada mais de uma vez

In [163]:
%time viral, v_members, _ = build(df[df['viral']], v=True)

- Total de mensagens: 110091
- Número de membros:
  - Ativos: 3487
  - Total: 3487
- Número de grupos: 225

CPU times: user 4.36 s, sys: 22.7 ms, total: 4.38 s
Wall time: 4.38 s


#### Grafo Desinformação

In [164]:
%time misinformation, m_members, _ = build(df[df['misinformation']], v=True)

- Total de mensagens: 8749
- Número de membros:
  - Ativos: 1647
  - Total: 1647
- Número de grupos: 175

CPU times: user 756 ms, sys: 3.84 ms, total: 760 ms
Wall time: 758 ms


## Part IV - Conexão com o Neo4J e Criação dos Grafos

```sh
# Start do container
$ cd /path/to/work
$ docker-compose -up -d neo4j
```

<div class="alert alert-block alert-info">
    A versão community do Neo4J não deixa fácil existirem vários databases, então eu joguei todos os databases em um único grafo com arestas de tipos diferentes.
</div>

In [165]:
# Installs - Necessário porque meu Jupyter está em um Docker
!pip install py2neo

# Imports
from py2neo import Graph

In [166]:
uri = "bolt://neo4j:7687"      # Deve coincidir com os valores informados no docker-compose.yml. Nesse caso o nome do serviço.
username = "neo4j"             # O Neo4j exige esse usuário
password = "12345678"          # O Neo4j exige uma senha de 8 dígitos

#
g = Graph(uri, auth=(username, password))
# session = g.session(database="neo4j") # Caso você tenha a versão paga

#### Nós

In [173]:
def create_nodes(driver, members):
    query = f'''UNWIND [
      {','.join(map(lambda m: f'{{name: "{m}", grau: "{members[m]["stats"]["geral"]["grau_centralidade"]}", forca: "{members[m]["stats"]["geral"]["forca"]}"}}', members))}
    ] AS membro
    
    CREATE (m:Membro {{name: membro.name, grau: membro.grau, forca: membro.forca}})
    '''

    _ = driver.run(query)
    return 

##### Desinformação

In [177]:
# Criando os membros
%time create_nodes(g, misinformation['senders'])

CPU times: user 7.33 ms, sys: 5.08 ms, total: 12.4 ms
Wall time: 830 ms


<div class="alert alert-block alert-info">
    Verificar com <code>MATCH(n) RETURN n LIMIT 50</code>
</div>

![](./files/images/neo4j-1.png)
![](./files/images/neo4j-2.png)

#### Arestas

In [187]:
def create_edges(driver, graph, start=0, v=False):
    count = start
    step = len(graph['senders'])//10
    next = start + 1
    
    for s in graph['senders'].keys():
        msgs = graph['senders'][s]['msgs']

        for r in msgs:
            query = f'''
                MATCH(r:Membro {{name: "{s}"}})
                MATCH(s:Membro {{name: "{r}"}})
                CREATE (r)-[e:MSGs {{total: {msgs[r]}}}]->(s)
            '''

            _ = driver.run(query)

        # Verbose: Para saber se está rodando :-)
        if v:
            count += 1
            if count == next:
                print(f'{count:04d}/{len(graph["senders"])} ~ ({count/len(graph["senders"])*100:.2f})%')
                next += step

##### Desinformação

In [186]:
# Misinformation
%time _ = create_edges(g, misinformation, v=True)

0001/1647 ~ (0.06)%
0165/1647 ~ (10.02)%
0329/1647 ~ (19.98)%
0493/1647 ~ (29.93)%
0657/1647 ~ (39.89)%
0821/1647 ~ (49.85)%
0985/1647 ~ (59.81)%
1149/1647 ~ (69.76)%
1313/1647 ~ (79.72)%
1477/1647 ~ (89.68)%
1641/1647 ~ (99.64)%
CPU times: user 9.96 s, sys: 1.8 s, total: 11.8 s
Wall time: 3min 54s


<div class="alert alert-block alert-info">
    Verificar com <code>MATCH(n) RETURN n LIMIT 300</code><br/>
    Disclaimer: Minha máquina não exibe mais que isso
</div>

![](./files/images/neo4j-3.png)

#### Backup do banco

```sh
#              CONTAINER                         DB
$ docker exec -it neo4j neo4j-admin database dump neo4j --to-path=/dumps/
```

### Parte V - Apresentação dos stats
> Monte uma tabela contendo a quantidade de nós e a quantidade de arestas para cada grafo
(mensagens gerais, mensagens virais e mensagens com desinformação).

In [139]:
geral['stats']['type'] = 'geral'
viral['stats']['type'] = 'viral'
misinformation['stats']['type'] = 'misinformation'

#
pd.DataFrame([geral['stats'], viral['stats'], misinformation['stats']])

,arestas,nos,geral,viral,misinformation,type
0,25087510,6165,186316,110091,8749,geral
1,6925768,3487,110091,110091,5181,viral
2,416799,1647,8749,5181,8749,misinformation
